# Using the diffusion equation to calculate equivalent resistance
## Ohm's law:
$$ \vec{J} = \sigma \vec{E} $$
$$ \vec{E} = \nabla V $$
$\vec{J}$ : Electric current density

$\vec{E}$ : Electric field

$V$ : Electric potential

## Steady State 
The continuity equation without source/sink:
$$ \nabla \cdot \vec{J} = 0 $$
This is the differential version of the Kirchoff's law.

Combining the above two condition:
$$ \nabla ( \sigma \nabla V ) = 0 $$

With an assumption that $ V \propto \rho $, 
the diffusion equation can be used to solve for the steady state numerically.

## Boundary Condition
* up-down : closed boundary
  * No current in or out through these boundaries
* left-right : open boundary
  * the voltages at the left and right boundary is set to 0 and 1
    * $ V_{left} = 0 $
    * $ V_{right} = 1 $
  * Current at boundaries : 
    * $ I_{left} = \sum_{i=left} \delta V_i/R_i $
    * $ I_{right} = \sum_{i=right} \delta V_i/R_i $
    * $ I_{left} = I_{right} $
* Equivalent Resistance = $ \frac{V_{right}-V_{left}}{(I_{left}+I_{right})/2} $

---
## Setup matplotlib backend

In [1]:
## embedded interactive plot (e.g.: jupyter notebook)
# %matplotlib notebook
## embedded interactive plot (e.g.: jupyter notebook)
## spawn a separate window (e.g.: jupyter lab)
%matplotlib qt
## spawn a separate window (e.g.: jupyter lab)

---
## Import everything required

In [2]:
## src.diffusion is dependent on the path
import src.diffusion as dfsn
## src.diffusion is dependent on the path
## src.current is dependent on the path
import src.current as curr
## src.current is dependent on the path
import numpy as np

---
## Diffusion for Voltages
Simulates the electric potential with:
* constant boundary voltages in one direction
* and open boundary everywhere else.

In [3]:
potential = curr.Resistor(n=(10,10), bc=['open','closed'])
potential.bc

[[1, 1], [0, 0]]

In [4]:
f_start = 0
f_at_boundary = [0, 1, 0, 0]
Resistances = np.ones(100).reshape(10,10)+0.0
# Resistances = np.ones((10,10))

In [5]:
potential.initialize(f=f_start, f_at_b=f_at_boundary, R=Resistances)
potential.f.T

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
potential.steadyState(precision=0.00000000001)

Simulating (ddt=0.125000): t=0.000000-->402.875000 - Steady State reached!


In [8]:
(potential.f[1:]-potential.f[:-1])[0,1:-1],(potential.f[1:]-potential.f[:-1])[-1,1:-1]

(array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]),
 array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]))

In [9]:
potential.D[1,1:-1],potential.D[-2,1:-1]

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [10]:
def effective_Resistance(pot, dir):
    pot_dim = len(pot.f.shape)
    R_in_b = [slice(1,-1) if dim!=dir else 1 for dim in range(pot_dim)]
    R_out_b = [slice(1,-1) if dim!=dir else -2 for dim in range(pot_dim)]
    V_in_b = [slice(1,-1) if dim!=dir else 0 for dim in range(pot_dim)]
    V_out_b = [slice(1,-1) if dim!=dir else -1 for dim in range(pot_dim)]
    V_diff = np.diff(pot.f,axis=dir)
    I_in = np.sum(V_diff[tuple(V_in_b)]/pot.D[tuple(R_in_b)])
    I_out = np.sum(V_diff[tuple(V_out_b)]/pot.D[tuple(R_out_b)])
    del_V = np.mean(pot.f[tuple(V_out_b)] - pot.f[tuple(V_in_b)])
    
    R_eff = np.abs(del_V/((I_in+I_out)/2))
    print(f"R_eff = {R_eff}")

In [11]:
effective_Resistance(potential,dir=0)

R_eff = 2.0


---
## Animated 2D diffusion

In [12]:
%%time
from importlib import reload
reload(dfsn)
reload(curr)
potential = curr.Resistor(n=(20,20), bc=['open','closed'])
f_at_boundary = [0, 1, 0, 0]
f_start = 0.5
Resistances = (np.random.randint(2,size=potential.n))*10+1
potential.initialize(f=f_start, f_at_b=f_at_boundary, R=Resistances)
with curr.Animator(simObj=potential, savefile="data/potSS2D.gif", figsize=(9,9)) as plotter:
    plotter.start_animation(simulate="steadyState", kwargs={'precision':0.00000001, 'plot_dt_steps':1000})

Simulating (ddt=0.011364) : t=0.000000-->1106.295455 - Steady State reached!
Animation saved to: data/potSS2D.gif
Wall time: 33.9 s


In [16]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()

In [17]:
plt.ion()
ax.cla()
ax.plot(potential.f[1,1:-1]-potential.f[0,1:-1])
ax.plot(potential.f[-1,1:-1]-potential.f[-2,1:-1])
ax.plot((potential.f[1,1:-1]-potential.f[0,1:-1])*(potential.D[1,1:-1])/2)
ax.plot((potential.f[-1,1:-1]-potential.f[-2,1:-1])*(potential.D[-2,1:-1])/2)
fig.canvas.draw()
plt.pause(0.01)
plt.show()

In [18]:
np.sum((potential.f[-1,1:-1]-potential.f[-2,1:-1])/(potential.D[-2,1:-1]))/2,\
np.sum((potential.f[1,1:-1]-potential.f[0,1:-1])/(potential.D[1,1:-1]))/2

(0.05987007269643149, 0.05979674982246269)

In [19]:
effective_Resistance(potential,dir=0)

R_eff = 8.356535077565965


In [20]:
potential.save_checkpoint(filename="data/pot_chkpt")

Saved to: data/pot_chkpt.json & data/pot_chkpt.npy


In [21]:
potential.load_checkpoint(filename="data/pot_chkpt")

Loaded: data/pot_chkpt.json & data/pot_chkpt.npy
